### 思路1

&emsp;&emsp;在BPTT算法中,将误差项不断进行展开有:

\begin{align}
\mathbf{\delta}_{t, k} &=\frac{\partial \mathcal{L}_{t}}{\partial \mathbf{z}_{k}} \\
              &=\frac{\partial \mathbf{h}_{k}}{\partial \mathbf{z}_{k}} \frac{\partial \mathbf{z}_{k+1}}{\partial \mathbf{h}_{k}} \frac{\partial \mathcal{L}_{t}}{\partial \mathbf{z}_{k+1}} \\
&=
\begin{bmatrix}
\frac{ \partial f([z_{k}]_1)}{\partial [z_{k}]_1} & 0 & \cdots & 0  \\
0 & \frac{ \partial f([z_{k}]_2)}{\partial [z_{k}]_2} & \cdots & 0 \\
\vdots& \vdots & \ddots & \vdots \\
0 & 0 & \cdots & \frac{ \partial f([z_{k}]_{-1})}{\partial [z_{k}]_{-1}}
\end{bmatrix}  W_{hh}^{T} \boldsymbol{\delta_{\mathit{t, k+1}}} \\ \\\
&=A_k W_{hh}^{\top} \boldsymbol{\delta_{\mathit{t, k+1}}} \\ \\\
&=\prod_{s=k}^{t-1} \left( A_s W_{hh}^{\top} \right) \boldsymbol{\delta}_{t,t}
\end{align}

如果定义$\gamma = || A_s W_{hh}^{\top} ||$,则

$$ \boldsymbol{\delta}_{t,k} \approx \gamma^{t-k} \boldsymbol{\delta}_{t,t} $$

&emsp;&emsp;若$\gamma>1$,当$t-k \rightarrow \infty$时,$\gamma^{t-k}  \rightarrow  \infty $.当间隔$t-k$比较大时,梯度也会变得很大,
会造成系统不稳定,称为梯度爆炸问题.

&emsp;&emsp;若$\gamma<1$,当$t-k \rightarrow \infty$时,$\gamma^{t-k}  \rightarrow  0 $.当间隔$t-k$比较大时,梯度也会变得非常小,
会出现和深层前馈神经网络类似的梯度消失问题.

&emsp;&emsp;值得注意的是,在循环神经网络中的梯度消失不是说$\frac{\partial \mathcal{L}}{\partial W_{hh}}$的梯度消失了,而是$ \frac{\partial \mathcal{L}}{\partial \mathbf{h}_{k}}$(原因:$ \frac{\partial \mathcal{L}}{\partial \mathbf{h}_{k}} = \frac{\partial \mathbf{z}_{k+1}}{\partial \mathbf{h}_{k}}  \frac{\partial \mathcal{L}}{\partial \mathbf{z}_{k+1}} = W_{hh} \frac{\partial \mathcal{L}}{\partial \mathbf{z}_{k+1}} $)的梯度消失了(当间隔$t-k$比较大时).也就是说,参数$W_{hh}$的更新主要靠当前时刻$t$的几个相邻状态$\mathbf{h}_{k}$(由循环神经网络更新公式易知)来更新,长距离的状态对参数$W_{hh}$没有影响

### 思路2

&emsp;&emsp;假设一个非常简单的,缺少非线性激活函数和输入$\mathbf{x}$的循环神经网络为

\begin{align}
\mathbf{h}_t &= W_{hh} \mathbf{h}_{t-1} \\
&=(W_{hh})^t \mathbf{h}_0 \tag{1}
\end{align}

若$W_{hh}$符合下列形式的特征分解

$$ W_{hh} = Q \Lambda Q^{-1}$$

其中$Q$可逆,故式(1)可进一步写为

$$ \mathbf{h}_t = Q \Lambda^t Q^{-1}\mathbf{h}_0  $$

特征值提升到$t$次后,导致幅值不到一的特征值衰减到零,而幅值大于一的就会激增.从而出现梯度小时或梯度爆炸问题.

### 长期依赖问题
&emsp;&emsp;由于循环神经网络经常使用非线性激活函数为Logistic函数或Tanh函数作为非线性激活函数,其导数值都小于$1$,
并且权重矩阵$W_{hh}$也不会太大,因此如果时间间隔$t-k$过大,$\mathbf{\delta}_{t, k} $会趋向于0,因而经常会出现梯度消失问题.

&emsp;&emsp;虽然简单循环网络理论上可以建立长时间间隔的状态之间的依赖关系,但是由于梯度爆炸或消失问题,实际上只能学习到短期的依赖关系.
这样,如果时刻$t$的输出$y_t$依赖于时刻𝑘的输入$\mathbf{x}_k$,当间隔$t-k$比较大时,简单神经网络很难建模这种长距离的依赖关系,
称为长程依赖问题(Long-Term Dependencies Problem).